# Modélisation du prix des maisons

Ce notebook documente l'ensemble du flux de travail : exploration des données, préparation, entraînement de modèles et génération d'une soumission réutilisable pour l'application Streamlit. Les cellules Markdown détaillent chaque étape afin de servir de rapport méthodologique complet.

## 1. Importation des librairies et configuration
Nous commençons par charger les packages nécessaires pour l'analyse exploratoire et la modélisation.

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import joblib

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

## 2. Chargement des données
Nous utilisons les fichiers `train.csv` et `test.csv` fournis. La variable cible est `SalePrice` dans l'échantillon d'entraînement.

In [ ]:
DATA_DIR = pathlib.Path('.')
train_path = DATA_DIR / 'train.csv'
test_path = DATA_DIR / 'test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_df.shape, test_df.shape

## 3. Aperçu des données
Un premier coup d'œil sur les colonnes principales permet d'identifier rapidement la structure du jeu de données.

In [ ]:
train_df.head()

In [ ]:
train_df.describe(include='all').transpose().iloc[:15]

## 4. Données manquantes
Nous calculons le pourcentage de valeurs manquantes par colonne et visualisons les variables les plus touchées.

In [ ]:
missing_ratio = (train_df.isnull().mean() * 100).sort_values(ascending=False)
missing_ratio.head(15)

In [ ]:
top_missing = missing_ratio.head(15).sort_values()
ax = top_missing.plot(kind='barh', color='steelblue')
ax.set_title('Top 15 des colonnes avec valeurs manquantes (%)')
ax.set_xlabel('Pourcentage de valeurs manquantes')
plt.tight_layout()

## 5. Analyse de la variable cible
`SalePrice` présente une distribution asymétrique ; nous observons également sa version log-transformée pour stabiliser la variance.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(train_df['SalePrice'], kde=True, ax=axes[0], color='forestgreen')
axes[0].set_title('Distribution de SalePrice')
sns.histplot(np.log1p(train_df['SalePrice']), kde=True, ax=axes[1], color='darkorange')
axes[1].set_title('Distribution de log(SalePrice + 1)')
plt.tight_layout()

## 6. Colonnes numériques et catégorielles
Nous séparons les variables numériques et catégorielles afin de définir des pipelines de prétraitement adaptés.

In [ ]:
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = [col for col in train_df.columns if col not in categorical_cols + ['SalePrice']]

print(f"Variables numériques ({len(numeric_cols)}): {numeric_cols[:10]}{'...' if len(numeric_cols) > 10 else ''}")
print(f"Variables catégorielles ({len(categorical_cols)}): {categorical_cols[:10]}{'...' if len(categorical_cols) > 10 else ''}")

## 7. Corrélations avec la cible
Nous observons les variables numériques les plus corrélées avec `SalePrice` pour guider l'interprétation.

In [ ]:
corr_matrix = train_df[numeric_cols + ['SalePrice']].corr()
top_corr = corr_matrix['SalePrice'].abs().sort_values(ascending=False).head(12).index

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix.loc[top_corr, top_corr], annot=True, fmt='.2f', cmap='crest')
plt.title('Corrélations des variables numériques avec SalePrice')
plt.tight_layout()

## 8. Prétraitement
- **Numérique :** imputation par la médiane.
- **Catégoriel :** imputation par la modalité la plus fréquente, puis encodage *one-hot* avec gestion des modalités inconnues.

La séparation entre `X` et `y` est réalisée avant la division en apprentissage/test.

In [ ]:
X = train_df.drop(columns=['SalePrice'])
y = train_df['SalePrice']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_cols),
        ('categorical', categorical_transformer, categorical_cols),
    ]
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_valid.shape

## 9. Fonction d'évaluation
Pour comparer les modèles, nous utilisons la RMSE (erreur quadratique moyenne racine), la MAE et le coefficient $R^2$.

In [ ]:
def evaluate(model, X_tr, y_tr, X_te, y_te):
    pred_train = model.predict(X_tr)
    pred_test = model.predict(X_te)

    metrics = {
        'RMSE_train': mean_squared_error(y_tr, pred_train, squared=False),
        'MAE_train': mean_absolute_error(y_tr, pred_train),
        'R2_train': r2_score(y_tr, pred_train),
        'RMSE_test': mean_squared_error(y_te, pred_test, squared=False),
        'MAE_test': mean_absolute_error(y_te, pred_test),
        'R2_test': r2_score(y_te, pred_test),
    }
    return pd.Series(metrics)

## 10. Modèle 1 : Random Forest Regressor
Forêt aléatoire robuste aux relations non linéaires et interactions. Nous évaluons ses performances sur l'échantillon de validation.

In [ ]:
rf_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', RandomForestRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    ))
])

rf_model.fit(X_train, y_train)
rf_scores = evaluate(rf_model, X_train, y_train, X_valid, y_valid)
rf_scores

## 11. Modèle 2 : XGBoost Regressor
Gradient boosting performant sur données tabulaires. Les hyperparamètres sont calibrés pour un bon compromis biais/variance.

In [ ]:
xgb_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror'
    ))
])

xgb_model.fit(X_train, y_train)
xgb_scores = evaluate(xgb_model, X_train, y_train, X_valid, y_valid)
xgb_scores

## 12. Comparaison des modèles
Nous rassemblons les métriques clés pour sélectionner le modèle final.

In [ ]:
pd.DataFrame({'RandomForest': rf_scores, 'XGBoost': xgb_scores})

## 13. Importance des variables du meilleur modèle
Nous extrayons les importances issues du modèle XGBoost (meilleur score attendu) en tenant compte des variables encodées.

In [ ]:
def get_feature_importance(model, numeric, categorical, top_n=20):
    preprocess_step = model.named_steps['preprocess']
    ohe = preprocess_step.named_transformers_['categorical'].named_steps['encoder']
    feature_names = numeric + list(ohe.get_feature_names_out(categorical))
    booster = model.named_steps['model']
    importances = booster.feature_importances_
    importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    return importance_df.sort_values(by='importance', ascending=False).head(top_n)

importance_df = get_feature_importance(xgb_model, numeric_cols, categorical_cols, top_n=25)
ax = importance_df.sort_values('importance').plot.barh(x='feature', y='importance', color='indianred')
ax.set_title('Top 25 - Importance des variables (XGBoost)')
plt.tight_layout()
importance_df

## 14. Entraînement sur l'ensemble des données et export des artefacts
Nous ré-entraînons le modèle XGBoost sur tout l'échantillon d'entraînement, générons une prédiction pour `test.csv` et sauvegardons :
- le modèle complet (`mon_deuxieme_model.joblib`)
- l'encodeur One-Hot (`one_hot_encoder.joblib`)
- la liste des features encodées (`features_list.pkl`)
- le fichier de soumission (`submission.csv`).

In [ ]:
# Entraînement final
xgb_final = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror'
    ))
])

xgb_final.fit(X, y)

# Préparation du test et prédiction
test_predictions = xgb_final.predict(test_df)
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': test_predictions
})
submission.to_csv('submission.csv', index=False)

# Sauvegarde des artefacts pour l'application Streamlit
joblib.dump(xgb_final, 'mon_deuxieme_model.joblib')
encoder = xgb_final.named_steps['preprocess'].named_transformers_['categorical'].named_steps['encoder']
joblib.dump(encoder, 'one_hot_encoder.joblib')

# Liste des features après encodage
encoded_feature_names = numeric_cols + list(encoder.get_feature_names_out(categorical_cols))
import pickle
with open('features_list.pkl', 'wb') as f:
    pickle.dump(encoded_feature_names, f)

submission.head()

## 15. Points clés et prochaines étapes
- La pipeline de prétraitement gère automatiquement les valeurs manquantes et l'encodage catégoriel.
- XGBoost offre de meilleures performances et est retenu comme modèle principal.
- Les artefacts sauvegardés sont prêts pour une utilisation dans l'interface Streamlit.

Pour aller plus loin, on pourrait explorer :
- L'optimisation d'hyperparamètres (par validation croisée).
- L'ajout de variables dérivées (qualité moyenne des pièces, âge du bâtiment, etc.).
- L'analyse d'erreurs ciblée pour identifier les propriétés mal estimées.